# Import FGVC Aircraft database

In [15]:
!curl -O https://www.robots.ox.ac.uk/~vgg/data/fgvc-aircraft/archives/fgvc-aircraft-2013b.tar.gz
!tar xzf fgvc-aircraft-2013b.tar.gz
!mv fgvc-aircraft-2013b dataset 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2625M  100 2625M    0     0  29.2M      0  0:01:29  0:01:29 --:--:-- 29.8M


# Imports libraries

In [1]:
# baselib 
import pathlib

# imports 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import yaml

#from
from sklearn.model_selection import train_test_split
#from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

from PIL import Image

# Define constants

In [3]:
from google.colab import files
uploaded = files.upload()

Saving config.yaml to config.yaml


In [4]:
for key, value in uploaded.items():
  constants = yaml.load(value)
  print(key)  # prints the file name
  print(constants) # prints the loaded data

config.yaml
{'DATA_DIR': 'content/dataset/data', 'MODELS_DIR': 'content/models', 'TARGET_NAME': 'family', 'IMAGE_WIDTH': 128, 'IMAGE_HEIGHT': 128, 'IMAGE_DEPTH': 3}


In [5]:
DATA_DIR = constants['DATA_DIR']
MODELS_DIR = constants['MODELS_DIR']
TARGET_NAME = constants['TARGET_NAME']
IMAGE_WIDTH = constants['IMAGE_WIDTH']
IMAGE_HEIGHT = constants['IMAGE_HEIGHT']
IMAGE_DEPTH = constants['IMAGE_DEPTH']

# Preliminaries

In [6]:
def build_image_database(path, target):
  """Build a pandas dataframe with target class and access path to images.

  Parameters
  ----------
  path (Path): path pattern to read csv file containing images information.
  target (str): name of the target column.

  Returns
  -------
  A pandas dataframe, including target class and path to image.
  """
  _df = pd.read_csv(path, sep='\t',
              names=['all'],
              dtype={'all': str},              # ids are not int but string
            )

  # La fonction split() découpe une chaîne de caractères
  _df['image_id'] = _df['all'].apply(lambda x: x.split(' ')[0])

  # La fonction '<car>'.join(liste) concatène les éléments de liste en utilisant le séparateur <car>
  _df[target] = _df['all'].apply(lambda x: ' '.join(x.split(' ')[1:]))

  # La colonne path contient le chemin d'accès à l'image
  _df['path'] = _df['image_id'].apply(lambda x: pathlib.Path('dataset/data/images') / (x + '.jpg'))

  return _df.drop(columns=['all'])

In [7]:
def show_image(df, row, target):
  """Show an image from an image database, with the associated class.

  Parameters
  ----------
  df (pd.DataFrame): images definition dataframe
  row (int): row index in df of image to be displayed
  target (str): name of the target column

  Returns
  -------
  None
  """
  assert target in df.columns, "Missing target column in dataframe"
  assert 'path' in df.columns, "Missing image path in dataframe"
  print(df.iloc[row,][target])
  plt.imshow(plt.imread(df.iloc[row,]['path']))
  return

In [8]:
def load_resize_image(path, height, width):
  """Load an image and resize it to the target size.

  Parameters
  ----------
  path (Path): access path to image file
  height (int): resize image to this height
  width (int): resize image to this width
  
  Returns
  -------
  np.array containing resized image
  """
  return np.array(Image.open(path).resize((width, height)))



In [9]:
def build_x_and_y(df: pd.DataFrame, target: str, images: str):
  """Build x tensor and y tensor for model fitting.

  Parameters
  ----------
  df (pd.DataFrame): dataframe containing images and target
  target (str): name of target column
  images (str): name of images column

  Returns
  -------
  x (np.array): tensor of x values
  y (np.array): tensor of y values
  """
  x = np.array(df[images].to_list())
  y = tf.keras.utils.to_categorical(df[target].astype('category').cat.codes)
  return x, y

# Define the model

In [10]:
def build_classification_model(df: pd.DataFrame, target: str, images: str):
  """Build a TF model using information from target and images columns in dataframe.

  Parameters
  ----------
  df (pd.DataFrame): dataframe with target and images columns
  target (str): column name for target variable
  images (str): column name for images

  Returns
  -------
  TF model built & compiled
  """
  nb_classes = df[target].nunique() # Compute number of classes for output layer
  size = df[images].iloc[0].shape # Compute images size for input layer

  #Building the model
  model = Sequential()
  model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=size))
  model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
  model.add(MaxPool2D(pool_size=(2, 2)))
  model.add(Dropout(rate=0.25))
  model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
  model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
  model.add(MaxPool2D(pool_size=(2, 2)))
  model.add(Dropout(rate=0.25))
  model.add(Flatten())
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(rate=0.5))
  model.add(Dense(nb_classes, activation='softmax')) # output layer with nb_classes

  #Compilation of the model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  return model

# Classification of images

In [11]:
def classify_images(images, model, classes_names=None):
  """Classify images through a TF model.

  Parameters
  ----------
  images (np.array): set of images to classify
  model (tf.keras.Model): TF/Keras model
  classes_names: dictionary with names of classes

  Returns
  -------
  predicted classes
  """

  results = model.predict(images) # predict for images
  classes = np.argmax(results, axis=1) # argmax returns the index of the max value per row
  if classes_names is not None:
    classes = np.array(classes_names[classes])
  return classes

# Save the model

In [12]:
def save_model(model, basename):
  """Save tf/Keras model.

  Model file is named model + timestamp.

  Parameters
  ----------
  model (tf/Keras model): model to be saved
  basename: location to save model file
  """
  model.save('{}_{}.h5'.format(basename, datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')))
  return

# Initialisation of the TPU

In [13]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Running on TPU  ['10.77.133.2:8470']
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.77.133.2:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.77.133.2:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


# Train the model with target name

In [16]:
# Call preliminaires fonctions to get train and test dataset
path_train = 'images_' + TARGET_NAME + '_train.txt'
path_test = 'images_' + TARGET_NAME + '_test.txt'

df_train = build_image_database(path ='/' + DATA_DIR + '/' + path_train, target = TARGET_NAME)
df_test = build_image_database(path = '/' + DATA_DIR + '/' + path_test, target = TARGET_NAME)

# Resize images to get 128*128*3 for every images
df_train['resized_image']=df_train.apply(lambda x: load_resize_image(x['path'],IMAGE_HEIGHT,IMAGE_WIDTH),axis=1)
df_test['resized_image']=df_test.apply(lambda x: load_resize_image(x['path'],IMAGE_HEIGHT,IMAGE_WIDTH),axis=1)

# Build x and y for both train and test set
X_train, y_train = build_x_and_y(df_train, TARGET_NAME,'resized_image')
X_test, y_test = build_x_and_y(df_test, TARGET_NAME,'resized_image')


In [20]:
%%time

with tpu_strategy.scope():
  model = build_classification_model(df_train, TARGET_NAME, 'resized_image')

model.summary()



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 124, 124, 32)      2432      
                                                                 
 conv2d_5 (Conv2D)           (None, 120, 120, 32)      25632     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 60, 60, 32)       0         
 2D)                                                             
                                                                 
 dropout_3 (Dropout)         (None, 60, 60, 32)        0         
                                                                 
 conv2d_6 (Conv2D)           (None, 58, 58, 64)        18496     
                                                                 
 conv2d_7 (Conv2D)           (None, 56, 56, 64)        36928     
                                                      

In [21]:
epochs = 10
history = model.fit(X_train,y_train, batch_size = 100, epochs=epochs,  validation_data =(X_test,y_test))

Epoch 1/10
34/34 [==============================] - 35s 696ms/step - loss: 35.1006 - accuracy: 0.0372 - val_loss: 4.2100 - val_accuracy: 0.0795
Epoch 2/10
34/34 [==============================] - 3s 83ms/step - loss: 4.1749 - accuracy: 0.0711 - val_loss: 4.1465 - val_accuracy: 0.0792
Epoch 3/10
34/34 [==============================] - 3s 88ms/step - loss: 4.0515 - accuracy: 0.0798 - val_loss: 3.9804 - val_accuracy: 0.0840
Epoch 4/10
34/34 [==============================] - 3s 83ms/step - loss: 3.8967 - accuracy: 0.0957 - val_loss: 3.9315 - val_accuracy: 0.0987
Epoch 5/10
34/34 [==============================] - 3s 84ms/step - loss: 3.7367 - accuracy: 0.1020 - val_loss: 3.7875 - val_accuracy: 0.1122
Epoch 6/10
34/34 [==============================] - 3s 83ms/step - loss: 3.5486 - accuracy: 0.1416 - val_loss: 3.7501 - val_accuracy: 0.1302
Epoch 7/10
34/34 [==============================] - 3s 82ms/step - loss: 3.3729 - accuracy: 0.1617 - val_loss: 3.7049 - val_accuracy: 0.1290
Epoch 8/10

# Save the model

In [22]:
model.save(MODELS_DIR +"/"+ TARGET_NAME + ".h5")
